In [2]:
#neural network model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv('../../../data/m16_cpu_usage_v2_10minutes_anomaly_v3_with_lag.csv')

In [3]:
df['time'] = pd.to_datetime(df['time'])

In [4]:
df

,time,machine_id,cpu_util,anomaly,lag_1,lag_2
0,2017-10-03 00:20:00,m16,0.253333,0,0.340000,0.300000
1,2017-10-03 00:30:00,m16,0.200000,0,0.253333,0.340000
2,2017-10-03 00:40:00,m16,0.200000,0,0.200000,0.253333
3,2017-10-03 00:50:00,m16,0.248333,0,0.200000,0.200000
4,2017-10-03 01:00:00,m16,0.200000,0,0.248333,0.200000
...,...,...,...,...,...,...
3246,2017-10-25 13:20:00,m16,0.285000,0,0.200000,0.270000
3247,2017-10-25 13:30:00,m16,0.456667,0,0.285000,0.200000
3248,2017-10-25 13:40:00,m16,0.200000,0,0.456667,0.285000
3249,2017-10-25 13:50:00,m16,0.263333,0,0.200000,0.456667


In [5]:
X = df[['cpu_util', 'lag_1', 'lag_2']]
y = df['anomaly']

In [6]:
X_train = X[:int(0.8*len(X))]
X_test = X[int(0.8*len(X)):]
y_train = y[:int(0.8*len(y))]
y_test = y[int(0.8*len(y)):]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [7]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_train_scaled

In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(3, 1), return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


82/82 [==============================] - 6s 20ms/step - loss: 0.2623 - accuracy: 0.9446 - val_loss: 0.2200 - val_accuracy: 0.9892
Epoch 2/10
82/82 [==============================] - 1s 7ms/step - loss: 0.2070 - accuracy: 0.9446 - val_loss: 0.0759 - val_accuracy: 0.9892
Epoch 3/10
82/82 [==============================] - 1s 7ms/step - loss: 0.1982 - accuracy: 0.9446 - val_loss: 0.0568 - val_accuracy: 0.9892
Epoch 4/10
82/82 [==============================] - 1s 7ms/step - loss: 0.1877 - accuracy: 0.9450 - val_loss: 0.0415 - val_accuracy: 0.9892
Epoch 5/10
82/82 [==============================] - 1s 7ms/step - loss: 0.1699 - accuracy: 0.9454 - val_loss: 0.0399 - val_accuracy: 0.9892
Epoch 6/10
82/82 [==============================] - 1s 7ms/step - loss: 0.1645 - accuracy: 0.9492 - val_loss: 0.0307 - val_accuracy: 0.9923
Epoch 7/10
82/82 [==============================] - 1s 7ms/step - loss: 0.1590 - accuracy: 0.9500 - val_loss: 0.0314 - val_accuracy: 0.9908
Epoch 8/10
82/82 [============

In [45]:
predict = model.predict([[100.123, 61.213, 53.123]])
predict

1/1 [==============================] - 0s 53ms/step


array([[0.54669684]], dtype=float32)

In [23]:
if predict > 0.52:
    print("anomaly")
else:
    print("normal")

anomaly


In [26]:
model.save('model.h5')

In [3]:

df_test = pd.read_csv('../../generated_data/generated_cpu_data_test.csv')

In [4]:
df_test

,cpu_util,date
0,66.834766,2023-07-25 00:03:06.466
1,82.067813,2023-07-25 00:08:06.466
2,100.895190,2023-07-25 00:13:06.466
3,73.259342,2023-07-25 00:18:06.466
4,82.504274,2023-07-25 00:23:06.466
...,...,...
1435,17.553220,2023-07-29 23:38:06.568
1436,36.001713,2023-07-29 23:43:06.568
1437,21.256935,2023-07-29 23:48:06.568
1438,24.075016,2023-07-29 23:53:06.568


In [6]:
#add lag
num_lags = 2
for i in range(1, num_lags+1):
    df_test['lag_{}'.format(i)] = df_test['cpu_util'].shift(i)
    
df_test = df_test.dropna()

In [7]:
df_test

,cpu_util,date,lag_1,lag_2
2,100.895190,2023-07-25 00:13:06.466,82.067813,66.834766
3,73.259342,2023-07-25 00:18:06.466,100.895190,82.067813
4,82.504274,2023-07-25 00:23:06.466,73.259342,100.895190
5,62.700953,2023-07-25 00:28:06.466,82.504274,73.259342
6,88.266038,2023-07-25 00:33:06.466,62.700953,82.504274
...,...,...,...,...
1435,17.553220,2023-07-29 23:38:06.568,50.442835,49.355296
1436,36.001713,2023-07-29 23:43:06.568,17.553220,50.442835
1437,21.256935,2023-07-29 23:48:06.568,36.001713,17.553220
1438,24.075016,2023-07-29 23:53:06.568,21.256935,36.001713


In [8]:
predict = model.predict(X_test)

NameError: name 'model' is not defined